In [ ]:
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import cv2
import os
import numpy as np
from sklearn.metrics import classification_report

images=[]
labels=[]

def augment_images(image_array,labelName):
    global images
    global labels

    def rotate_image(image, angle):
        rows, cols = image.shape[:2]
        rotation_matrix = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
        rotated_image = cv2.warpAffine(image, rotation_matrix, (cols, rows))
        return rotated_image

    def flip_image(image, flip_code):
        flipped_image = cv2.flip(image, flip_code)
        return flipped_image

    def mirror_image(image):
        mirrored_image = cv2.flip(image, -1)
        return mirrored_image

    for image in image_array:
        images.append(image)
        labels.append(labelName)
        images.append(rotate_image(image, 90))
        labels.append(labelName)
        images.append(rotate_image(image, 180))
        labels.append(labelName)
        images.append(rotate_image(image, 270))
        labels.append(labelName)
        images.append(flip_image(image, 1))
        labels.append(labelName)
        images.append(rotate_image(flip_image(image, 1), 90))
        labels.append(labelName)
        images.append(rotate_image(flip_image(image, 1), 180))
        labels.append(labelName)
        images.append(rotate_image(flip_image(image, 1), 270))
        labels.append(labelName)


def read_jpg_images(folder_path):
    temp_img = []

    for filename in os.listdir(folder_path):
        if filename.endswith('.jpg') or filename.endswith('.jpeg'):
            image_path = os.path.join(folder_path, filename)
            image = Image.open(image_path)
            image = image.convert('L')
            pixel_array = np.array(image)
            temp_img.append(pixel_array)

    return temp_img

jpg_folder_path_pos = '/content/drive/MyDrive/dataset JPG/25th of all/Pos'
unhealthy_img = read_jpg_images(jpg_folder_path_pos)

jpg_folder_path_neg = '/content/drive/MyDrive/dataset JPG/25th of all/Neg/'
healthy_img = read_jpg_images(jpg_folder_path_neg)

print("The length of unhealthy input images is ",len(unhealthy_img))
print("The length of healthy input images is ",len(healthy_img))

augment_images(unhealthy_img,"unhealthy")
augment_images(healthy_img,"healthy")

print("The length of images is ",len(images))
print("The length of labels is ",len(labels))

label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)


desired_width = 227
desired_height = 227

def preprocess_images(images):
    processed_images = np.array([np.array(Image.fromarray(img).resize((desired_width, desired_height))) for img in images])
    processed_images = processed_images.astype('float32') / 255.0
    processed_images = processed_images.reshape(-1, desired_width, desired_height, 1)
    return processed_images

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(desired_width, desired_height, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

train_images, test_val_images, train_labels, test_val_labels = train_test_split(images, labels, test_size=0.3)

test_images, val_images, test_labels, val_labels = train_test_split(test_val_images, test_val_labels, test_size=0.7)

train_images = preprocess_images(train_images)
val_images = preprocess_images(val_images)
test_images = preprocess_images(test_images)

model.fit(train_images, train_labels, epochs=10, validation_data=(val_images, val_labels))

test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


The length of unhealthy input images is  30
The length of healthy input images is  30
The length of images is  480
The length of labels is  480
Epoch 1/10
11/11 [==============================] - 2s 63ms/step - loss: 1.3817 - accuracy: 0.5685 - val_loss: 0.6894 - val_accuracy: 0.4950
Epoch 2/10
11/11 [==============================] - 0s 39ms/step - loss: 0.6670 - accuracy: 0.6101 - val_loss: 0.6224 - val_accuracy: 0.6733
Epoch 3/10
11/11 [==============================] - 0s 39ms/step - loss: 0.4455 - accuracy: 0.8661 - val_loss: 0.2716 - val_accuracy: 0.9109
Epoch 4/10
11/11 [==============================] - 0s 41ms/step - loss: 0.1291 - accuracy: 0.9613 - val_loss: 0.1867 - val_accuracy: 0.9208
Epoch 5/10
11/11 [==============================] - 0s 39ms/step - loss: 0.0151 - accuracy: 1.0000 - val_loss: 0.0956 - val_accuracy: 0.9604
Epoch 6/10
11/11 [==============================] - 0s 40ms/step - loss: 0.0015 - accuracy: 1.0000 - val_loss: 0.0945 - val_accuracy: 0.9703
Epoch 7/10

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
